![schema.png](schema.png)

In [57]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

## R - receptance, K - key, V - value

### блок WKV (Weighted Key Value) играет ключевую роль в механизме внимания, использует выходы из слоев Key и Value. Эти выходы затем используются для вычисления взвешенного суммарного значения (Weighted Sum)

![formula.png](formula.png)

![formula2.png](formula2.png
)

![formula3.png](formula3.png)

In [58]:
class RWKV_Time_Mixing(torch.nn.Module):
    def __init__(self, embedding_dim, head_dim, num_heads, ctx_len):
        super().__init__()
        self.mu = nn.ReLU() # ??? Какая-то функция активации мю. ПОКА ЗАМЕНИЛА НА RELU
        self.Receptance = nn.Linear(embedding_dim, num_heads * head_dim)
        self.Key = nn.Linear(embedding_dim, num_heads * head_dim)
        self.Value = nn.Linear(embedding_dim, num_heads * head_dim)
        self.output = nn.Linear(num_heads * head_dim, embedding_dim)
        
        self.num_heads = num_heads # Кол-во голов
        self.head_dim = head_dim # Размер головы
        self.ctx_len = ctx_len

        # Взяла с гита, сложна
        with torch.no_grad(): # initial time_w curves for better convergence
            ww = torch.ones(self.num_heads, self.ctx_len)
            curve = torch.tensor([-(self.ctx_len - 1 - i) for i in range(self.ctx_len)]) # the distance
            for h in range(self.num_heads):
                if h < self.num_heads - 1:
                    decay_speed = math.pow(self.ctx_len, -(h + 1)/(self.num_heads - 1))
                else:
                    decay_speed = 0.0
                ww[h] = torch.exp(curve * decay_speed)
                # print('layer', layer_id, 'head', h, 'decay_speed', round(decay_speed, 4), ww[h][:5].numpy(), '...', ww[h][-5:].numpy())
        self.time_w = nn.Parameter(ww)
        self.time_alpha = nn.Parameter(torch.ones(self.num_heads, 1, self.ctx_len))
        self.time_beta = nn.Parameter(torch.ones(self.num_heads, self.ctx_len, 1))
        self.time_gamma = nn.Parameter(torch.ones(self.ctx_len, 1))
        self.time_shift = nn.ZeroPad2d((0, 0 , 1 , -1))

    def forward(self, x):
        B, T, C = x.size()

        # Тоже взяла, сложна
        TT = self.ctx_len
        w = F.pad(self.time_w, (0, TT))
        w = torch.tile(w, [TT])
        w = w[:, :-TT].reshape(-1, TT, 2 * TT - 1)
        w = w[:, :, TT-1:] # w is now a circulant matrix
        w = w[:, :T, :T] * self.time_alpha[:, :, :T] * self.time_beta[:, :T, :]

        x = torch.cat([self.time_shift(x[:, :, :C//2]), x[:, :, C//2:]], dim = -1)

        #--------

        x = self.mu(x) # (batch_size, seq_len, embedding_dim)
        r = torch.sigmoid(self.Receptance(x)) # (batch_size, seq_len, head_dim)
        k = self.Key(x) # (batch_size, seq_len, head_dim)
        v = self.Value(x) # (batch_size, seq_len, head_dim)

        # Из гитхаба
        # k = torch.clamp(k, max=30, min=-60) # clamp extreme values. e^30 = 10^13
        # k = torch.exp(k)
        # sum_k = torch.cumsum(k, dim=1)
        # kv = (k * v).view(B, T, self.n_head, self.head_size)
        # wkv = (torch.einsum('htu,buhc->bthc', w, kv)).contiguous().view(B, T, -1)
        # rwkv = torch.sigmoid(r) * wkv / sum_k
        # rwkv = self.output(rwkv)

        k = torch.exp(k) # Возводим Key в экспоненту для сумм в Attm+(W, K, V) ()
        sum_k = torch.cumsum(k, dim = 1) # Кумулятивная сумма по , (batch_size, seq_len, head_dim)
        kv = (k * v).view(B, T, self.num_heads, self.head_dim) # матричное уможение
        wkv = (torch.einsum('htu,buhc->bthc', w, kv)).contiguous().view(B, T, -1) # непонятный какой-то этот ваш einsum
        rwkv = wkv / sum_k # ну делим числитель на знаменатель
        rwkv = self.output(rwkv)

        return rwkv

In [59]:
class RWKV_Channel_Mixing(torch.nn.Module):
   def __init__(self, embedding_dim, hidden_dim):
      super().__init__()
      self.mu = nn.ReLU() # опять эта фигня. ПОКА ЗАМЕНИЛА НА RELU
      self.Receptance = nn.Linear(embedding_dim, embedding_dim)
      self.Key = nn.Linear(embedding_dim, hidden_dim)
      self.Value = nn.Linear(hidden_dim, embedding_dim)

   def forward(self, x):
      B, T, C = x.size()
      x = self.mu(x)
      r = torch.sigmoid(self.Receptance(x))
      kv = self.Value(self.Key(x))
      return (r * kv).view(B, T, -1)

In [60]:
class RWKV_Block(nn.Module):
    def __init__(self, embedding_dim, num_heads, head_dim, ctx_len, hidden_dim):
        super().__init__()

        self.Layer_Norm1 = nn.LayerNorm(embedding_dim)
        self.Layer_Norm2 = nn.LayerNorm(embedding_dim)

        self.timemix = RWKV_Time_Mixing(embedding_dim, head_dim, num_heads, ctx_len)
        self.channelmix = RWKV_Channel_Mixing(embedding_dim, hidden_dim)

    def forward(self, x):
        x = x + self.timemix(self.Layer_Norm1(x))
        x = x + self.channelmix(self.Layer_Norm2(x))
    
        return x

In [61]:
class RWKV_LM_Head(nn.Module):
    def __init__(self, embedding_dim, n_class):
        super().__init__()

        self.Layer_Norm = nn.LayerNorm(embedding_dim)
        self.output = nn.Linear(embedding_dim, n_class)

    def forward(self, x):
        x = self.output(self.Layer_Norm(x))
    
        return x

In [62]:
class RWKV_model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_heads, head_dim, ctx_len, hidden_dim, num_layers, n_class):
        super().__init__()
        # Объединить это всё надо
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.Layer_Norm = nn.LayerNorm(embedding_dim)
        self.blocks = nn.Sequential(*[RWKV_Block(embedding_dim, num_heads, head_dim, ctx_len, hidden_dim) for i in range(num_layers)])
        self.lm_head = RWKV_LM_Head(embedding_dim, n_class)

    def forward(self, x):
        x = self.embedding(x)
        x = self.Layer_Norm(x)
        x = self.blocks(x)
        x = self.lm_head(x)
        return x

# Подготовка датасета

In [63]:
# hyperparameters
batch_size = 64
block_size = 4
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6 # D = 384 // 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

In [64]:
!wget https://raw.githubusercontent.com/marulyanova/NLP_6sem/main/dataset_poetry_mac.txt

--2024-05-27 15:34:37--  https://raw.githubusercontent.com/marulyanova/NLP_6sem/main/dataset_poetry_mac.txt
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 677788 (662K) [text/plain]
Сохранение в: «dataset_poetry_mac.txt.5»

dataset_poetry_mac. 100%[===================>] 661,90K  4,10MB/s    за 0,2s    

2024-05-27 15:34:37 (4,10 MB/s) - «dataset_poetry_mac.txt.5» сохранён [677788/677788]



In [65]:
import re

with open('/Users/maria/Documents/NLP6sem/Project RWKV/dataset_poetry_mac.txt', 'r', encoding = 'MACCYRILLIC') as file:
    lines = file.readlines()

new_lines = []
for line in lines:

    # убрать пробелы в начале строки, оставить только русские буквы, убрать строки, где только цифры (года написания стихов), названия стихов

    line = line.lstrip()
    line = re.sub(r'[a-zA-Z]', '', line)
    if line.isdigit():
        continue
    if line.isupper():
            line = '*\n'
    new_lines.append(line)

with open('dataset_poetry_mac_modified.txt', 'w', encoding = 'MACCYRILLIC') as file:
    file.writelines(new_lines)

with open('dataset_poetry_mac_modified.txt', 'r', encoding = 'MACCYRILLIC') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [66]:
text[:600].split('\n')

['Ты опять упрекнула меня,',
 'Что я с музой моей раздружился,',
 'Что заботам текущего дня',
 'И забавам его подчинился.',
 'Для житейских расчетов и чар',
 'Не расстался б я с музой моею,',
 'Но бог весть, не погас ли тот дар,',
 'Что, бывало, дружил меня с нею?',
 'Но не брат еще людям поэт,',
 'И тернист его путь, и непрочен,',
 'Я умел не бояться клевет,',
 'Не был ими я сам озабочен;',
 'Но я знал, чье во мраке ночном',
 'Надрывалося сердце с печали',
 'И на чью они грудь упадали свинцом,',
 'И кому они жизнь отравляли.',
 'И пускай они мимо прошли,',
 'Надо мною ходившие грозы,',
 'Знаю я, чьи молитвы и слезы',
 'Роковую стрелу отвели...',
 'Да и время ушло,- я устал']

In [67]:
len(text) # длина датасета 600к символов

626213

In [68]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [69]:
@torch.no_grad()
def estimate_loss():
    criterion = nn.CrossEntropyLoss()
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits = model(X)
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            Y = Y.view(B * T)
            loss = criterion(logits, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Модель

In [70]:
model = RWKV_model(vocab_size, n_embd, n_head, n_embd, block_size, n_embd, n_layer, vocab_size).to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

24.024361 M parameters


In [71]:
try:
  optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
  criterion = nn.CrossEntropyLoss()

  for iter in range(max_iters):

      if iter % eval_interval == 0 or iter == max_iters - 1:
          losses = estimate_loss()
          print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

      xb, yb = get_batch('train')
      logits = model(xb)
      B, T, C = logits.shape
      logits = logits.view(B * T, C)
      yb = yb.view(B * T)
      loss = criterion(logits, yb)
      optimizer.zero_grad(set_to_none = True)
      loss.backward()
      optimizer.step()

except KeyboardInterrupt: 0

step 0: train loss 4.6921, val loss 4.6823


## Пример CUMsum по разным осям

In [8]:
# Вторая размерность (1) - сложение по i-му элементу по второму измерению))
a = torch.tensor([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]]) 
torch.cumsum(a, dim = 1), torch.cumsum(a, dim = 1).shape

# У нас в модели вот так, т.е. сложение по seq_len элементов head_dim (выходы складываются друг с другом по i-му эл-ту, покоординатно)

(tensor([[[ 1,  2,  3],
          [ 5,  7,  9]],
 
         [[ 7,  8,  9],
          [17, 19, 21]]]),
 torch.Size([2, 2, 3]))

In [10]:
# Первая размерность (0) - сложение по i-му элементу по первому измерению
a = torch.tensor([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]]) 
torch.cumsum(a, dim = 0), torch.cumsum(a, dim = 0).shape

(tensor([[[ 1,  2,  3],
          [ 4,  5,  6]],
 
         [[ 8, 10, 12],
          [14, 16, 18]]]),
 torch.Size([2, 2, 3]))

In [11]:
# Самая последняя размерность (2) - просто по порядку складываем чиселки.
a = torch.tensor([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]]) 
torch.cumsum(a, dim = 2), torch.cumsum(a, dim = 2).shape

(tensor([[[ 1,  3,  6],
          [ 4,  9, 15]],
 
         [[ 7, 15, 24],
          [10, 21, 33]]]),
 torch.Size([2, 2, 3]))